<a href="https://colab.research.google.com/github/mad-scientist-in-training/BIOF509-Final-Project/blob/Project-Code/BIOF509_Final_Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#This is a project to determine if speech patterns in Parkinson's patients vs healthy controls can be used to train a Support Vector Machine model to predict which patients belong to which category.

In [4]:
#Code outline

In [7]:
#Import necessary libraries and packages
import numpy as np
import pandas as pd
import seaborn
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
#manipulate/visualize dataset to keep features you are interested in and get rid of ones you don't care about



In [ ]:
#split the data into training and test sets

In [ ]:
#create the knn model

In [ ]:
#fit the model to the training data

In [ ]:
#carry out model on testing data

In [ ]:
#calculate score for how well the model fits